In [3]:
import pickle 
fraud_lat = []
fraud_lon = [] 
nofraud_lat = [] 
nofraud_lon = [] 

with open("fraud_lat.bin", "rb") as f: 
    fraud_lat = pickle.load(f)

with open("fraud_lon.bin", "rb") as f: 
    fraud_lon = pickle.load(f)
    
with open("nofraud_lat.bin", "rb") as f: 
    nofraud_lat = pickle.load(f)
    
with open("nofraud_lon.bin", "rb") as f: 
    nofraud_lon = pickle.load(f) 


    


In [7]:
import pandas as pd 
df = pd.read_csv('data.csv') 

# Not fraud dataset
nofraud = df[df['status'] != 'Fraud'][0:5000]

#Fraud dataset 
fraud = df[df['status'] == 'Fraud'][0:5000]

nofraud['lat'] = nofraud_lat[0:5000]
nofraud['lon'] = nofraud_lon[0:5000]
fraud['lat'] = fraud_lat[0:5000] 
fraud['lon'] = fraud_lon[0:5000]

In [9]:
# Normalize lon and lat variables 
from sklearn import preprocessing
import numpy as np

x = fraud.lon #returns a numpy array
x = np.array(x).reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fraud['lon'] = x

x = fraud.lat #returns a numpy array
x = np.array(x).reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fraud['lat'] = x

x = nofraud.lon #returns a numpy array
x = np.array(x).reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
nofraud['lon'] = x

x = nofraud.lat #returns a numpy array
x = np.array(x).reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
nofraud['lat'] = x

In [11]:
import time 
import datetime 

seconds_in_day = 24*60*60

fraud_timestamps = [] 
nofraud_timestamps = [] 

for i in fraud.timestamp:
    fraud_timestamps.append(time.mktime(datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S.%f").timetuple()))

for i in nofraud.timestamp:
    nofraud_timestamps.append(time.mktime(datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S.%f").timetuple()))


nofraud['secs'] = nofraud_timestamps
fraud['secs'] = fraud_timestamps

fraud['sin_time'] = np.sin(2*np.pi*fraud.secs/seconds_in_day)
fraud['cos_time'] = np.cos(2*np.pi*fraud.secs/seconds_in_day)
nofraud['sin_time'] =  np.sin(2*np.pi*nofraud.secs/seconds_in_day)
nofraud['cos_time'] =  np.cos(2*np.pi*nofraud.secs/seconds_in_day)

In [12]:
from sklearn.utils import shuffle


# create dataset for training and testing 
frames = [fraud, nofraud]
df = pd.concat(frames)
df = df[['lon', 'lat', 'sin_time', 'cos_time', 'valid']]
df = shuffle(df)
df['valid'] = df['valid'].astype(int)

train = df.iloc[0:7000]
test = df.iloc[7000:10000]

train_x = train.iloc[:, 0:4]
train_y = train.iloc[:, 4:5]

test_x = test.iloc[:,0:4] 
test_y = test.iloc[:,4:5]

In [21]:
from sklearn.linear_model import LinearRegression
x = train_x.to_numpy() 
y = train_y.to_numpy()

reg = LinearRegression().fit(x, y)
reg.score(x,y)



0.6633147190452996

In [22]:

x = test_x.to_numpy()
y = test_y.to_numpy() 
reg.score(x,y)

0.6739930534450412

In [24]:
df.to_pickle("frames.bin")

In [25]:
train = df.iloc[0:7000]
test = df.iloc[7000:10000]

train_x = train.iloc[:, 0:4]
train_y = train.iloc[:, 4:5]

test_x = test.iloc[:,0:4] 
test_y = test.iloc[:,4:5]